# Delta Lake Deep Dive - Spark Hands-on Notebook

This notebook accompanies Chapter 5 and demonstrates how to use Delta Lake with Apache Spark. Please run each code block in sequence for a step-by-step exploration of Delta Lake features.



## Environment Setup

- **Recommended**: Use the provided Docker Compose environment for a reproducible setup.
- Or, use a Spark 3.x and Delta Lake 2.x+ local install with:

```bash
pip install pyspark==3.4.1 delta-spark==2.4.0 findspark
```
- Java 8+ is required for Spark.
- The remainder of this notebook will run inside a suitable Jupyter or Databricks environment.



In [1]:
# Check Spark & Delta import
import pyspark
from delta import *
print("PySpark version:", pyspark.__version__)



PySpark version: 3.4.1


## Table of Contents
- Delta Table Creation (SQL & DataFrame)
- Inserts & Queries
- Update, Delete, Merge (Upsert)
- Schema Evolution
- Time Travel
- Change Data Feed (CDC)
- Optimization (Z-Ordering, Vacuum)
- Streaming Reads/Writes



## 1. Delta Table Creation with Spark

Let's start by creating a Delta table using both SQL and the DataFrame API. We'll use a local file path for simplicity. Ensure the working directory has the right permissions, or update the path as needed.



In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder.appName('DeltaLakeDemo')
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel('WARN')

print('Spark session started!')


Spark session started!


In [3]:
# Create a Delta table using SQL DDL
spark.sql('DROP TABLE IF EXISTS delta_test')
spark.sql('''
    CREATE TABLE delta_test (
        id INT, name STRING, amount FLOAT
    ) USING DELTA
''')
print('Created Delta table delta_test!')


Created Delta table delta_test!


In [4]:
# Create a Delta table using the DataFrame API
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('amount', FloatType(), True)
])
df = spark.createDataFrame([], schema)
df.write.format('delta').mode('overwrite').save('/tmp/delta_table_df')
print('Delta table created at /tmp/delta_table_df')


Delta table created at /tmp/delta_table_df


## 2. Insert Records and Basic Queries

Let's insert records into our Delta tables and show how to query the data using both SQL and the DataFrame API.


In [5]:
# Insert records using SQL
spark.sql('''
    INSERT INTO delta_test VALUES
        (1, 'Alice', 120.5),
        (2, 'Bob', 85.0),
        (3, 'Carol', 310.8)
''')
print('3 records inserted into delta_test!')

# Query records using SQL
spark.sql('SELECT * FROM delta_test').show()


3 records inserted into delta_test!
+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  1|Alice| 120.5|
|  2|  Bob|  85.0|
+---+-----+------+



In [6]:
# Insert records using DataFrame API (append to /tmp/delta_table_df)
data = [
    (4, 'David', 220.1),
    (5, 'Eve', 99.5)
]
df2 = spark.createDataFrame(data, schema)
df2.write.format('delta').mode('append').save('/tmp/delta_table_df')
print('Appended 2 records to /tmp/delta_table_df')

# Query DataFrame-based Delta table
df_read = spark.read.format('delta').load('/tmp/delta_table_df')
df_read.show()


Appended 2 records to /tmp/delta_table_df
+---+-----+------+
| id| name|amount|
+---+-----+------+
|  4|David| 220.1|
|  5|  Eve|  99.5|
+---+-----+------+



---

You should now see your inserted Delta Lake records both via SQL and DataFrame API. Next, we'll explore updates, deletes, and upserts (MERGE INTO) with Delta tables.
p

## 3. Update, Delete, and Upsert (MERGE) Operations

Delta Lake supports powerful transactional DML statements. Let's perform row-level `UPDATE`, `DELETE`, and upsert with `MERGE INTO`.


In [7]:
# Update a row by SQL
spark.sql("UPDATE delta_test SET amount = amount + 20 WHERE id = 2")
spark.sql("SELECT * FROM delta_test").show()


+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  1|Alice| 120.5|
|  2|  Bob| 105.0|
+---+-----+------+



In [8]:
# Delete a row by SQL
spark.sql("DELETE FROM delta_test WHERE id = 1")
spark.sql("SELECT * FROM delta_test").show()


+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  2|  Bob| 105.0|
+---+-----+------+



In [9]:
# Upsert rows using MERGE INTO
# First, add a table with updates
spark.sql("DROP TABLE IF EXISTS delta_updates")
spark.sql('''
    CREATE TABLE delta_updates (
        id INT, name STRING, amount FLOAT
    ) USING DELTA
''')
spark.sql('''
    INSERT INTO delta_updates VALUES
        (2, 'Bob', 200.0),   -- UPDATE: id=2 will be updated
        (4, 'Daisy', 177.7)  -- INSERT: id=4 is new
''')

spark.sql('''
MERGE INTO delta_test t
USING delta_updates u
ON t.id = u.id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
''')
spark.sql('SELECT * FROM delta_test').show()


+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  2|  Bob| 200.0|
|  4|Daisy| 177.7|
+---+-----+------+



## 4. Schema Evolution: Add, Drop, Rename Columns

Delta Lake supports schema evolution, meaning you can add, drop, and rename columns on Delta tables easily. Let's walk through the major operations:


In [10]:
# Add a column
spark.sql("ALTER TABLE delta_test ADD COLUMNS (email STRING)")
spark.sql("DESCRIBE TABLE delta_test").show()


+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|      int|   null|
|    name|   string|   null|
|  amount|    float|   null|
|   email|   string|   null|
+--------+---------+-------+



In [11]:
# Upgrade the table to support drop/rename columns (enable column mapping by name)
spark.sql("""
ALTER TABLE delta_test SET TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name',
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5'
)
""")

DataFrame[]

In [12]:
# Drop a column
spark.sql("ALTER TABLE delta_test DROP COLUMN email")
spark.sql("DESCRIBE TABLE delta_test").show()


+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|      int|   null|
|    name|   string|   null|
|  amount|    float|   null|
+--------+---------+-------+



In [13]:
# Rename a column (Delta Lake >= 2.0.0)
spark.sql("ALTER TABLE delta_test RENAME COLUMN name TO customer_name")
spark.sql("DESCRIBE TABLE delta_test").show()


+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|           id|      int|   null|
|customer_name|   string|   null|
|       amount|    float|   null|
+-------------+---------+-------+



In [14]:
# Idempotent rename: 'name' -> 'customer_name' only if 'name' exists
cols = [f.name for f in spark.table('delta_test').schema.fields]
if 'name' in cols:
    spark.sql("ALTER TABLE delta_test RENAME COLUMN name TO customer_name")
else:
    print("No column 'name' to rename.")
spark.sql("DESCRIBE TABLE delta_test").show()



No column 'name' to rename.
+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|           id|      int|   null|
|customer_name|   string|   null|
|       amount|    float|   null|
+-------------+---------+-------+



In [15]:
# ... do other schema evolution steps or demo updates here if needed ...



In [16]:
# Idempotent rename: 'customer_name' -> 'name' only if 'customer_name' exists
cols = [f.name for f in spark.table('delta_test').schema.fields]
if 'customer_name' in cols:
    spark.sql("ALTER TABLE delta_test RENAME COLUMN customer_name TO name")
else:
    print("No column 'customer_name' to rename.")
spark.sql("DESCRIBE TABLE delta_test").show()



+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|      int|   null|
|    name|   string|   null|
|  amount|    float|   null|
+--------+---------+-------+



---

That's schema evolution! Add, drop, or rename columns safely—even after table creation. Next, let's see how Time Travel works in Delta Lake.


## 5. Time Travel (Querying Past Table Versions)

Delta Lake lets you query your table as it appeared at a previous version or timestamp! This enables rollback, auditing, and recovery after accidental changes.


In [17]:
# Show current table state
delta_df = spark.read.format('delta').table('delta_test')
delta_df.show()


+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  2|  Bob| 200.0|
|  4|Daisy| 177.7|
+---+-----+------+



In [18]:
# Suppress FutureWarning about Pandas datetime64 dtype for cleaner output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



### Delta Lake Version History

Below, we show the Delta Lake table's commit history. Note that version 0 is just the table creation: it won't have any data! Use a version **after the first INSERT** for meaningful time travel queries.


In [19]:
# Show commit/version history to find which versions have your data
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, 'delta_test')
dt.history().select('version', 'timestamp', 'operation').show(truncate=False)


+-------+-----------------------+-----------------+
|version|timestamp              |operation        |
+-------+-----------------------+-----------------+
|9      |2025-12-02 19:57:03.735|RENAME COLUMN    |
|8      |2025-12-02 19:57:03.43 |RENAME COLUMN    |
|7      |2025-12-02 19:57:03.145|DROP COLUMNS     |
|6      |2025-12-02 19:57:02.861|SET TBLPROPERTIES|
|5      |2025-12-02 19:57:02.553|ADD COLUMNS      |
|4      |2025-12-02 19:57:02.112|MERGE            |
|3      |2025-12-02 19:57:00.484|DELETE           |
|2      |2025-12-02 19:56:59.803|UPDATE           |
|1      |2025-12-02 19:56:58.273|WRITE            |
|0      |2025-12-02 19:56:54.91 |CREATE TABLE     |
+-------+-----------------------+-----------------+



### Time Travel Demo (Querying Past Table Versions)

Set `version_num` to a version after the CREATE TABLE (usually version 1 or later) to view data as it was after the first insert/update/merge.


In [20]:
# Get the current version
import delta
from delta.tables import DeltaTable

dt = DeltaTable.forName(spark, 'delta_test')
history_df = dt.history()
history_df.show()  # Shows operation history, version, and timestamp


+-------+--------------------+------+--------+-----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|        operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+-----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      9|2025-12-02 19:57:...|  null|    null|    RENAME COLUMN|{oldColumnPath ->...|null|    null|     null|          8|  Serializable|         true|                  {}|        null|Apache-Spark/3.4....|
|      8|2025-12-02 19:57:...|  null|    null|    RENAME COLUMN|{oldColumnPath ->...|null|    null|     null|          7|  Serializable|         true|                  {}|     

In [21]:
# Time travel best practice: Always inspect schema before queries!
old_df = spark.read.format('delta').option('versionAsOf', 0).table('delta_test')
# See which columns exist (use them when querying old_df)
old_df.printSchema()
# Now query with the actual columns at that version ('name', not 'customer_name')
old_df.select('id','name','amount').show()


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: float (nullable = true)

+---+----+------+
| id|name|amount|
+---+----+------+
+---+----+------+



In [22]:
# General pattern for time travel: inspect schema, then select existing columns
version_num = 1  # Set this to the version you want
tt_df = spark.read.format('delta').option('versionAsOf', version_num).table('delta_test')
print('Schema at Delta version', version_num, ':', tt_df.columns)
tt_df.show()  # Will always work, shows all columns that exist at that version

# Optionally, select specific columns as reported by printSchema()/columns
# Example: tt_df.select('id', 'customer_name', 'amount').show()

Schema at Delta version 1 : ['id', 'name', 'amount']
+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  1|Alice| 120.5|
|  2|  Bob|  85.0|
+---+-----+------+



In [23]:
# Query an older version of the Delta table (pick a previous version like 0 or 1)
old_df = spark.read.format('delta').option('versionAsOf', 2).table('delta_test')
old_df.show()

# Or query as of a timestamp (replace with suitable timestamp from history_df)
# old_df = spark.read.format('delta').option('timestampAsOf', '2024-01-01 00:00:00').table('delta_test')


+---+-----+------+
| id| name|amount|
+---+-----+------+
|  3|Carol| 310.8|
|  1|Alice| 120.5|
|  2|  Bob| 105.0|
+---+-----+------+



## 6. Change Data Feed (CDC)

Delta Lake supports Change Data Feed, letting you query only changed (insert/update/delete) rows between versions, if enabled (Delta Lake >= 2.0.0).


In [24]:
# Enable CDC feature on an existing table (newer Delta Lake only)
spark.sql("ALTER TABLE delta_test SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

# Get CDF between two versions. Pick versions that include some change events!
try:
    # You can use dt.history().show() to pick meaningful version ranges
    cdf = spark.read.format("delta") \
        .option("readChangeData", "true") \
        .option("startingVersion", 1) \
        .option("endingVersion", 3) \
        .table("delta_test")
    cdf.show()
except Exception as e:
    print("CDC not available between chosen versions or not supported: ", e)


CDC not available between chosen versions or not supported:  Error getting change data for range [1 , 3] as change data was not
recorded for version [1]. If you've enabled change data feed on this table,
use `DESCRIBE HISTORY` to see when it was first enabled.
Otherwise, to start recording change data, use `ALTER TABLE table_name SET TBLPROPERTIES
(delta.enableChangeDataFeed=true)`.


---

If CDF is not supported by your Delta version, this cell will error — just comment it out or upgrade Delta Lake as needed.

Now, let's optimize the Delta table using Z-Order and VACUUM.


In [25]:
# Robust CDC demo - only show if enabled and with real changes
from delta.tables import DeltaTable
import pandas as pd

dt = DeltaTable.forName(spark, 'delta_test')
hist_pd = dt.history().toPandas()
cdc_enabled_versions = hist_pd[
    hist_pd['operationParameters'].astype(str).str.contains('delta.enableChangeDataFeed', na=False)
]['version']

if not cdc_enabled_versions.empty:
    cdc_start = int(cdc_enabled_versions.values[0])
    print(f"CDC enabled at version {cdc_start}. We'll query after this version.")
    try:
        latest_ver = int(hist_pd['version'].max())
        if latest_ver > cdc_start:
            cdf = spark.read.format("delta") \
                .option("readChangeData", "true") \
                .option("startingVersion", cdc_start) \
                .option("endingVersion", latest_ver) \
                .table("delta_test")
            if cdf.count() == 0:
                print("CDC enabled, but no changes recorded between these versions.")
            else:
                cdf.show()
        else:
            print("CDC enabled, but no further versions with changes to display yet.")
    except Exception as e:
        print("CDC error: ", e)
else:
    print("CDC was never enabled for this table.")


CDC enabled at version 10. We'll query after this version.
CDC enabled, but no further versions with changes to display yet.


In [26]:
# Demonstrate CDC with real data: perform an operation after CDC is enabled
# Insert, update, or delete after enabling CDC so changes appear in the feed
spark.sql("INSERT INTO delta_test VALUES (200, 'AfterCDC', 500.0)")
spark.sql("UPDATE delta_test SET amount = amount + 999 WHERE id = 2")
spark.sql("DELETE FROM delta_test WHERE id = 3")

# Now rerun CDC (between CDC enable version and latest)
cdc_enabled_versions = hist_pd[
    hist_pd['operationParameters'].astype(str).str.contains('delta.enableChangeDataFeed', na=False)
]['version']
if not cdc_enabled_versions.empty:
    cdc_start = int(cdc_enabled_versions.values[0])
    latest_ver = int(hist_pd['version'].max())
    print(f"CDC enabled at version {cdc_start}, now querying changes up to {latest_ver}")
    try:
        cdf = spark.read.format("delta") \
            .option("readChangeData", "true") \
            .option("startingVersion", cdc_start) \
            .option("endingVersion", latest_ver) \
            .table("delta_test")
        cdf.show()
    except Exception as e:
        print("CDC error after data changes: ", e)
else:
    print("CDC was never enabled for this table.")



CDC enabled at version 10, now querying changes up to 10
+---+----+------+------------+---------------+-----------------+
| id|name|amount|_change_type|_commit_version|_commit_timestamp|
+---+----+------+------------+---------------+-----------------+
+---+----+------+------------+---------------+-----------------+



## 7. Optimization: Z-Order, VACUUM, and Table Maintenance

Delta Lake supports table optimization for query performance and storage cleanup.


In [27]:
# Optimize table data layout (requires Databricks or Delta OSS 2.0+ SQL)
# This command may not be available in open source Spark unless you use native OPTIMIZE extension.
try:
    spark.sql("OPTIMIZE delta_test ZORDER BY (customer_name)")
except Exception as e:
    print(f'OPTIMIZE not supported in this Spark. Skipping: {e}')


OPTIMIZE not supported in this Spark. Skipping: Z-Ordering column customer_name does not exist in data schema.


In [28]:
# Robust Z-Order OPTIMIZE: only run for columns that exist and if supported
cols = [f.name for f in spark.table('delta_test').schema.fields]
col_to_zorder = 'name' if 'name' in cols else cols[0]  # Use name if present, else first column
try:
    sql = f"OPTIMIZE delta_test ZORDER BY ({col_to_zorder})"
    print(f"Running: {sql}")
    spark.sql(sql)
except Exception as e:
    print(f'OPTIMIZE not supported in this Spark or on this data. Skipping: {e}')



Running: OPTIMIZE delta_test ZORDER BY (name)


In [29]:
# Remove old files no longer needed for time travel
spark.sql("VACUUM delta_test RETAIN 168 HOURS")  # retains files for 7 days by default


DataFrame[path: string]

---

VACUUM reclaims disk from unreferenced files (check your data retention policy before running it in production!).

Next: Streaming reads and writes with Delta Lake.


## 8. Streaming Reads and Writes

Delta Lake supports both batch (bounded) and streaming (continuous) data pipelines directly on tables.


In [30]:
# Set up a directory for file-based streaming demo
import os, shutil
stream_dir = "/tmp/delta_stream_source"
if os.path.exists(stream_dir):
    shutil.rmtree(stream_dir)
os.makedirs(stream_dir)

# Define the input schema to match written files
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
input_schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),  # use 'customer_name' if your schema is that
    StructField("amount", FloatType()),
])

# Start streaming read from that directory
streaming_input_df = spark.readStream.schema(input_schema).json(stream_dir)

# Start streaming write to Delta Lake table
query = streaming_input_df.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/delta_test_checkpoint") \
    .start("/tmp/delta_table_stream")

# Write a new file to the streaming source (simulates real-time event)
import json
data = {"id": 100, "name": "Fresh", "amount": 88.8}
with open(os.path.join(stream_dir, "test-event.json"), "w") as f:
    f.write(json.dumps(data))

import time
time.sleep(10)  # Let streaming job pick up the file and commit it
query.stop()

# Query the Delta table to see the new row
spark.read.format("delta").load("/tmp/delta_table_stream").show()


+---+-----+------+
| id| name|amount|
+---+-----+------+
|100|Fresh|  88.8|
+---+-----+------+



In [31]:
# Read from a Delta table as a stream
read_query = spark.readStream.format("delta").load("/tmp/delta_table_stream")
# To output the stream in the notebook, you would need to write it to a sink (console/parquet, etc.) in production.
# Here, we'll just show the DataFrame definition.
read_query.printSchema()


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: float (nullable = true)



---

That concludes the hands-on deep dive in Delta Lake with Spark!

- CRUD, schema evolution, DML/transactions, time travel, CDC, optimization, and streaming.

For more, refer to Delta Lake documentation or try more complex scenarios with your own datasets.
